# Ayudantía 09: Networking y webservices 

Ayudantes:

- Julio Huerta
- Felipe Vidal
- Diego Toledo
- Alejandro Held
- Clemente Campos

# Introducción
Como ya pudimos ver la semana pasada, la información en el computador es guardada en bytes. Esta forma de presentar la información es la más básica para los computadores y por tanto será la que ocuparemos para comunicarlos. En esta ayudantía repasaremos el concepto de neetworking, que es un concepto muy poderoso dentro de la computación que nos permite generar comunicación entre 2 o más computadores diferentes.

# Pequeño recordatorio

La semana pasada se abordaron algunos conceptos importantes para esta semana, los cuales se repasarán rapidamente

### IP
La ip es un número que identifica únicamente a un computador, es necesaria para poder entablar una conexión con otra máquina.

### Puerto
Dentro de un mismo computador existen miles de puertos. Cada puerto puede ser pedido por una aplicación distinta del computador, de esta forma puedo entablar comunicación precisamente con la aplicación que deseo.

# Envio de información

Empecemos con un ejemplo, digamos que quiero enviar un texto desde mi computador al computador de un amigo utilizando python ¿Como lo puedo hacer? Una forma es crear un **Servidor** el cual contenga el texto que quiero enviar y se enceuntre a la espera de la llegada de un cliente. Al momento de llegar el cliente deberá **Serializar** el texto que quiere enviar a **bytes** y luego enviarlo.

Es importante que al momento de conectar un cliente con un servidor, ambos programas estén de acuerdo en que método se utilizará para serializar la información. A continuación se verán 2 formas de serializar en python.

## Envio utilizando JSON

La semana anterior vimos que se puede serializar utilizando el formato JSON, el cual transforma la información a un string que después puede ser serializado utilizando bytes. Luego se podría enviar información de la siguiente forma.

In [1]:
import json
import socket

server_host = "127.0.0.1"    # Elegimos la ip del servidor destino
server_port = 12345


#Creamos el mensaje que queremos enviar
mensaje = {"titulo": "Don Quijote de la mancha", "año": 1605, "Texto": "En un lugar de la Mancha, de cuyo nombre no quiero acordarme..."}


# Paso 1) Pasamos el tipo de dato a un string
# Diccionario ---> string
mensaje = json.dumps(mensaje)
print(type(mensaje))

#Paso 2) Pasamos de un string a bytes eligiendo un encode.
# string ---> bytes
mensaje = mensaje.encode("utf-8")

#Utilizamos un socket para enviar la información
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((server_host, server_port))

#Paso 3) Enviamos el mensaje 
sock.sendall(mensaje)

#¿Como veo la información de la respuesta?
#Debido a que esperamos que lleguen bytes que representan un string en UTF-8 y que el string es un JSON podemos invertir el proceso

data = json.loads(sock.recv(4096).decode('utf-8'))
print("Respuesta servidor:" + str(data))
sock.close()

<class 'str'>


ConnectionRefusedError: [Errno 61] Connection refused

## Envio utilizando Pickle

Otra forma de serializar objetos de Python es **pickle** el cual es solo des-serializable por **otro programa de Python**. A continuación se repetirá el proceso anterior utilizando esta nueva forma de serializar

In [3]:
import pickle
import socket

server_host = "127.0.0.1"    # Elegimos la ip del servidor destino
server_port = 12345


#Creamos el mensaje que queremos enviar
mensaje = {"titulo": "Don Quijote de la mancha", "año": 1605, "Texto": "En un lugar de la Mancha, de cuyo nombre no quiero acordarme..."}


# Paso 1) Pasamos el tipo de dato a bytes directamente  ^^
# Diccionario ---> bytes 
mensaje = pickle.dumps(mensaje)

#Utilizamos un socket para enviar la información
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((server_host, server_port))

#Paso 2) Enviamos el mensaje 
sock.sendall(mensaje)

#¿Como veo la información de la respuesta?
#Debido a que esperamos que lleguen bytes que representan una serialización en Pickle, podemos devolver el proceso.
data = pickle.loads(sock.recv(10**5)) 
print("Respuesta servidor" + str(data))
sock.close()

Respuesta servidor:{'titulo': 'Don Quijote de la mancha', 'año': 1605, 'Texto': 'En un lugar de la Mancha, de cuyo nombre no quiero acordarme...'}


## Uso de chunck
En los ejemplos anteriores, al momento de recibir la respuesta se espera que el largo de la respuesta **sea menor a 4096** bytes. Pero el Quijote tiene un total de 381.104 palabras, por lo que no se podría leer el mensaje completo. Podriamos simplemente aceptar más bytes pero
* En caso de enviar mensajes con aún más bytes seguiría sin funcionar el código
* Tendría que procesar toda la información de golpe, cargando todo el Quijote en memoria RAM lo cual para archivos aún más grandes es una mala idea.

Una forma de aliviar esta carga sería dividir el mensaje en partes pequeñas. Cada una con un tamaño manejable para nuestro computador y que nos permita leer indefinidamente hasta que el mensaje acabe. Esta idea es un **Chunk**: dividir grandes cantidades de bytes trozos para enviar o recibir por partes.

Repitamos el proceso anteior con nuestra nueva herramienta.

In [ ]:
import pickle
import socket

server_host = "127.0.0.1"    # Elegimos la ip del servidor destino, en este caso es la nuetra
server_port = 12345


#Creamos el mensaje que queremos enviar
mensaje = {"titulo": "Don Quijote de la mancha", "año": 1605, "Texto": "En un lugar de la Mancha, de cuyo nombre no quiero acordarme..."}


# Paso 1) Pasamos el tipo de dato a bytes directamente  ^^
# Diccionario ---> bytes 
mensaje = pickle.dumps(mensaje)

#Utilizamos un socket para enviar la información
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((server_host, server_port))

#Paso 2) Enviamos el mensaje POR PARTES
inicio = 0
while inicio < len(mensaje):
    if inicio + 4096 < len(mensaje):
        chunck = mensaje[inicio: inicio + 4096]
    else:
        chunck = mensaje[inicio:]
    
    sock.sendall(chunck)

#¿Como veo la información de la respuesta?
#Debido a que esperamos que lleguen bytes por partes, recibimos todo y después unimos
respuesta = b""
while True:
    data = sock.recv(4096)  # Recibe un máximo de 4096 bytes
    if not data:
        break  # Sale del bucle si no hay más datos
    respuesta += data

diccionario = pickle.loads(respuesta)
print("Respuesta servidor:" + str(diccionario))
print(diccionario)

# Ejercicio Propuesto
Para mostrar el funcionanto de Networking avanzado, haremos una simulación de lo que sucede en una ayudantía, la cual consistirá de diversas partes:

1. `Estudiante`: Representa a un estudiante que hace preguntas en la ayudantía. Estas pueden ser de contenidos, de tarea o más dificiles. Se implementará con un cliente que busca conectarse a un servidor para poder realizar las preguntas

2. `Docencio`: Representa a un ayudante de docencia al que le llegan las preguntas de los alumnos en las ayudantías. El solo puede responder preguntas de contenido, las preguntas dificles y las de tarea se las delega a un ayudante Tareo. El `Docencio` Representara una clase que puede actuar tanto como de Cliente como de Servidor (P2P)

3. `Tareo`: Representa a un ayudante de tareas el cual responde aquellas preguntas sobre tareas, en caso de ser una pregunta dificil, se la hará llegar al profesor antes de dar una respuesta. Es implementado como un servidor que puede aceptar conexiones desde multiples clientes 

4. `Profesor`: Representa al profesor del ramo que contesta aquellas preguntas muy dificiles para los ayudantes de Tarea, ademas lleva el conteo de cuantas preguntas en total han sido contestadas. Será implementado como un servidor de Flask

Por lo tanto el flujo de preguntas y respuestas es de la siguiente forma: (Actualizar con un dibujo bonito)

![diagrama](diagrama.png)

# Cliente
Ahora formalicemos la idea de un cliente. Un cliente realiza conexiones a un servidor, este cliente realiza solicitudes a un servidor y busca **consumir sus servicios**. Por ejemplo al realizar una busqueda en Google, mi celular se comporta como cliente del servidor de Google porque busca consumir los servicios de Google.

A continuación se presenta un ejemplo básico de un cliente, posteriormente vamos a continuar complejizando este concepto. Notar que nuestro cliente quiere enviar mensajes a un servidor y además recibir respuestas. Luego, como cada una de estas acciones sucede en paralelo. A escuchar y a enviar mensajes les asignamos un **thread** distinto.

In [ ]:
from random import choice, random
from time import sleep
import threading

class Estudiante:
    def __init__(self, port: int, host: str) -> None:
        # Inicializa el cliente, establece la conexión al servidor y crea un hilo para hacer preguntas
        print("Inicializando cliente...")
        self.host = host
        self.port = port
        self.socket_client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        try:
            self.connect_to_server()
            self.listen()
            self.thread_preguntas = threading.Thread(target=self.consultar)
            self.thread_preguntas.start()  # Inicia las preguntas
            
        except ConnectionError:
            print("Conexión terminada.")
            self.socket_client.close()
            exit()

    def connect_to_server(self) -> None:
        # conecta el cliente al servidor
        self.socket_client.connect((self.host, self.port))
        print("Cliente conectado exitosamente al servidor.")


    def listen(self) -> None:
        # crea e inicializa un thread para comunicarse con el cliente
        thread = threading.Thread(target=self.listen_thread)
        thread.start()


    def send(self, msg: str) -> None:
        # codifica un str en bytes y lo envia al servidor
        msg_bytes = msg.encode()
        msg_length = len(msg_bytes).to_bytes(4, byteorder='big')
        self.socket_client.sendall(msg_length + msg_bytes)


    def listen_thread(self) -> None:
        # Thread que recibe respuestas del servidor
        while True:
            response_bytes_length = self.socket_client.recv(4)
            response_length = int.from_bytes(response_bytes_length, byteorder='big')
            response = bytearray()

            while len(response) < response_length:
                read_length = min(4096, response_length - len(response))
                response.extend(self.socket_client.recv(read_length))

            print(f"Respuesta recibida: {response.decode()}\n>>> ", end='')


    def consultar(self):
        while True:
            duda = choice(["Contenido", "Tarea", "Pregunta Dificil"])
            print(f"Alumno pregunta: {duda}")  # Imprimir tipo de pregunta
            self.send(duda)
            sleep(random() * 5 + 1)  # Espera entre 1 y 6 segundos

# Servidor Multicliente

Luego digamos que quiero hacer un servidor. Como se podrán imaginar un servidor como los de google reciben cientos de clientes por segundo, pero en los ejemplos de servidor vistos hasta ahora **el flujo del programa se detiene al hacer listen()**. Esto lleva a que solamente podamos estar trabajando con un cliente a la vez.

Entonces tenemos que poder recibir varios clientes, **comunicarnos con cada uno en paralelo** y de alguna manera ir **guardando el socket de cada cliente**. talvez suena a que podriamos ocupar... **THREADS**

**Solución**: Tendremos un Thread que solo se encarga de recibir clientes, al llegar un cliente creamos un nuevo thread que **solo se encarga de comunicarse con un alumno**

Así siempre pueden llegar nuevos clientes al servidor, además nuestro servidor siempre estará respondiendo a cada cliente. Perfecto



In [ ]:
import requests

class Tareo:
    def __init__(self, port: int, host: str) -> None:
        # inicializa el servidor y comienza a escuchar conexiones de clientes
        print("Inicializando servidor...")
        self.host = host
        self.port = port
        self.respuestas = [" No puedes usar esa libreria",
                            " No es un problema de los tests",
                            " Actualizamos los tests",
                            " Puedes usar esa librería :D"]
        
        self.socket_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.bind_and_listen()
        self.accept_connections()


    def bind_and_listen(self) -> None:
        # Enlaza el servidor al puerto especificado
        self.socket_server.bind((self.host, self.port))
        self.socket_server.listen()
        print(f"Servidor escuchando en {self.host}:{self.port}...")


    def accept_connections(self) -> None:
        # inicializa un thread para escuchar conexiones entrantes y aceptarlas
        thread = threading.Thread(target=self.accept_connections_thread)
        thread.start()


    def accept_connections_thread(self) -> None:
        # Función target que permitirá aceptar conexiones
        # crea un thread por cada cliente aceptado para poder escucharlo
        print("Servidor aceptando conexiones...")
        while True:
            client_socket, ip = self.socket_server.accept()
            listening_client_thread = threading.Thread(
                target=self.listen_client_thread,
                args=(client_socket, ))
            listening_client_thread.start()


    def send(self, value: any, sock: socket.socket) -> None:
        # Envía respuestas al cliente
        stringified_value = str(value)
        msg_bytes = stringified_value.encode()
        msg_length = len(msg_bytes).to_bytes(4, byteorder='big')
        sock.sendall(msg_length + msg_bytes)


    def listen_client_thread(self, client_socket: socket.socket) -> None:
        # Recibe preguntas desde el cliente, las procesa y las responde
        print("Servidor conectado a un nuevo cliente...")
        while True:
            # Recibe los bytes
            response_bytes_length = client_socket.recv(4)
            response_length = int.from_bytes(
                response_bytes_length, byteorder='big')
            response_length = int.from_bytes(response_bytes_length, byteorder='big')
            pregunta = client_socket.recv(response_length).decode()


            print(f"Recibimos una pregunta: {pregunta}")
            if pregunta == "Tarea":
                respuesta = choice(self.respuestas)
                self.send(respuesta, client_socket)
                print("Pregunta respondida al estudiante")
            

            elif pregunta == "Pregunta Dificil":
                # Pedir respuesta al profesor
                URL = "http://localhost:4444/respuesta"
                respuesta = requests.get(URL)
                status = respuesta.status_code
                print(f"CONSULTA AL PROFESOR: {status}")
                print(respuesta.json())
                self.send(respuesta.json()["texto"], client_socket)

# P2P

Veamos un caso más complejo. Digamos que yo tengo un servidor el cual espera la llegada de multiples clientes para poder entregar una respuesta. Pero a la vez, este servidor puede no ser capaz de resolver todas las solicitudes por si solo. Talvez, este servidor necesita **realizar consultas a otro servidor**. Luego el servidor a la vez debe ser un **Cliente**.

A estas situaciones donde se es cliente y servidor a la vez, se le conoce como P2P y llevan a pensar en arquitecturas **descentralizadas**. Si todos los miembros se comportan como servidor y cliente, entonces no existe un servidor central que dirija la red. Ejemplo de esto son redes para minar criptomonedas donde todos los miembros de la red son igual de importantes. Otro ejemplo son algunas aplicaciones de transferencia de archivos (*presuntamente piratas*) en que cada miembro tiene archivos descargados y en caso de que otro usuario lo necesite, puede solicitar el archivo al computador de otro miembro.

A continuación veremos un ejemplo de código. **No te asustes**, es un código extenso pero esto se debe a que tiene los métodos de un **servidor** y un **cliente**, no es excesivamente diferente a ninguno.

In [ ]:
import socket
import threading
import requests

class Docencia:
    # Inicia el cliente y el servidor
    def __init__(self, host: str, port: int, datos_otro_cliente: dict) -> None:
        print("Inicializando cliente P2P (2)...")
        self.host = host
        self.port = port
        self.datos_otro_cliente = datos_otro_cliente

        # Al ser P2P necesitamos un socket para ser cliente y un socket para ser servidor
        self.sockets = {
            'server': socket.socket(socket.AF_INET, socket.SOCK_STREAM),
            'client': socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        }
        self.bind_and_listen()
        self.accept_connections()

        try:
            self.connect_to_server()
        except ConnectionError:
            print("Conexión terminada.")
            self.sockets['client'].close()
            exit()


    def bind_and_listen(self) -> None:
        # enlaza y habilita el servidor para comenzar a escuchar conexiones entrantes
        self.sockets['server'].bind((self.host, self.port))
        self.sockets['server'].listen()
        print(f"Cliente P2P (2) escuchando en {self.host}:{self.port}...")


    def accept_connections(self) -> None:
        # Inicializa un thread que se encargara de aceptar conexiones entrantes
        thread = threading.Thread(target=self.accept_connections_thread)
        thread.start()

    def accept_connections_thread(self) -> None:
        # funcion target que acepta conexiones entrantes y les da un thread para poder escucharlos
        while True:
            client_socket, _ = self.sockets['server'].accept()
            threading.Thread(
                target=self.listen_client_thread,
                args=(client_socket,),
                daemon=True
            ).start()


    def listen_client_thread(self, client_socket: socket.socket) -> None:
        # funcion target que recibe el contenido del cliente y lo procesa para darle una respuesta
        while True:
            try:
                response_bytes_length = client_socket.recv(4)
                response_length = int.from_bytes(response_bytes_length, byteorder='big')
                pregunta = client_socket.recv(response_length).decode()

                print(f"Recibimos una pregunta: {pregunta}")

                if pregunta == "Contenido":
                    self.answer_client("Docencio: respondiendo pregunta de contenido", client_socket)
                else:
                    threading.Thread(
                        target=self.procesar_pregunta_tareo,
                        args=(pregunta, client_socket),
                        daemon=True
                    ).start()

            except ConnectionResetError:
                print("Conexión con el cliente cerrada.")
                break

    def procesar_pregunta_tareo(self, pregunta: str, client_socket: socket.socket) -> None:
        # Consulta a Tareo y responde al cliente con la respuesta obtenida
        respuesta = self.consultar_a_tareo(pregunta)
        self.answer_client(f"Respuesta del Tareo o Profesor: {respuesta}", client_socket)
        self.notificar_profesor()


    def answer_client(self, value: str, sock: socket.socket) -> None:
        # envia respuesta al cliente correspondiente
        msg_bytes = value.encode()
        sock.sendall(len(msg_bytes).to_bytes(4, byteorder='big') + msg_bytes)


    # Metodos del cliente
    def connect_to_server(self) -> None:
        # Conecta el Docencio (como cliente) al Tareo (el servidor)
        self.sockets['client'].connect(
            (self.datos_otro_cliente['host'], self.datos_otro_cliente['port'])
        )
        print("Cliente conectado exitosamente al otro cliente.")


    def consultar_a_tareo(self, pregunta: str) -> str:
        # Envia preguntas a tareo y espera la respuesta
        self.send(pregunta)
        response = self.sockets['client'].recv(4096).decode()
        return response

    def send(self, msg: str) -> None:
        # manda mensaje al Tareo (Servidor)
        msg_bytes = msg.encode()
        self.sockets['client'].sendall(len(msg_bytes).to_bytes(4, byteorder='big') + msg_bytes)


    def notificar_profesor(self) -> None:
        # Le indica al profesor (Server Flask) que ya contesto una pregunta correctamente
        try:
            response = requests.post("http://localhost:4444/preguntas")
            if response.status_code == 200:
                print(f"Contador de preguntas actualizado: {response.json()['texto']}")
        except requests.RequestException as e:
            print(f"Error al conectar con el profesor: {e}")

*Ejercicio propuesto: se puede reducir considerablemente el código utilizando de forma correcta la herencia*

# Webservices 
Ahora nos enfocaremos únicamente en el servidor. Un servidor, se puede entender como un *programa que realiza cierta acción* y que responde al cliente. Luego, para que la comunicación sea efectiva, ambas partes deben haber acordado como deben enviarse los mensajes pero entendiendo que el internet es una red enorme, se debe tener cierto **concenso** en como se debe hacer una solicitud a un servidor y como el servidor debe responder. 

Es aquí donde surge el concepto de protocolo de comunicación. El más importante es HTTP. Este permite la comuniación entre servidor y cliente de forma conocida para ambos miembros. El servidor que quiere ofrecer servicios expone ciertas rutas con ciertos métodos donde los clientes pueden entablar comunicación. Cada convinación de ruta y método cumple una misión distinta.

Dependiendo de lo que se quiera hacer se elige un métodos diferente. HTTP ofrece los siguientes métodos:
* `GET`: Se realiza cuando la solicitud busca obtener cierta información. Por ejemplo si quiero obtener la fotos que un amigo subio a instagram.

* `POST`: Se realiza cuando la solicitud busca publicar nueva información. Por ejemplo publicar una nueva foto en mi instagram.

* `PATCH`: Se realiza cuando la solicitud busca modificar **Solo parte de un recurso**. Por ejemplo cambiar la descripción de una fotografia en instagram. La foto se mantiene intacta pero solo una parte de la publicación se modifica.

* `PUT`: Se realiza cuando la solicitud **Cambia todalmente un recurso** por ejemplo cambiar la foto de perfil. El recurso de foto perfil se ve modificado totalmente.

* `DELETE`: Se realiza cuando la solicitud busca eliminar totalmente un recurso. Por ejemplo eliminar una publicación en instagram.

Además HTTP provee algunos códigos de estado que permiten saber el resultado de una solicitud.
* 2XX: La solicitud fue existosa.
* 4XX: Se produjo un problema del lado del cliente.
* 5XX: Se produjo un problema del lado del servidor.

Cada código significa algo distinto. Por ejemplo:
* 404: recurso no encontrado.
* 200: Todo correcto.

Una lista completa de los códigos HTTP (con fotos de gatitos) puede ser encontrado en [LINK](https://http.cat/).

A continuación se presenta una ejemplo de una API simple que tiene 3 rutas a las cuales se pueden acceder por otras aplicaciones.


In [1]:
from flask import Flask, request
from random import choice

app = Flask(__name__)

preguntas = 0  # Variable global para contar las preguntas

respuestas = ["No puedes usar esa libreria", "No es un problema de los tests",
              "Actualizamos los tests", "Puedes usar esa librería :D"]


@app.route("/", methods=["GET"])
def hello_world():
    return {"saludo": "Bienvenido a la clase"}


@app.route("/respuesta", methods=["GET"])
def respuesta():
    escogida = choice(respuestas)
    return {"texto": escogida, "método": "GET"}


@app.route("/preguntas", methods=["GET", "POST"])
def respuesta_ingeniosa():
    global preguntas  # Declaramos que usaremos la variable global
    
    if request.method == "POST":
        preguntas += 1
        return {"texto": f"Llevamos: {preguntas} preguntas respondidas", "método": "POST"}
    
    return {"texto": f"Llevamos: {preguntas} preguntas respondidas", "método": "GET"}

El Anterior codigo de servidor web fue subido a la nube a la plataforma de Render. Por lo cual es posible acceder a sus rutas remotamente, desde cualquier maquina. Puedes probarlo en las siguientes celdas:

In [1]:
import requests

respuesta = requests.get("https://profesor-flask.onrender.com/")
respuesta.json()

{'saludo': 'Bienvenido a la clase'}

In [2]:
respuesta = requests.get("https://profesor-flask.onrender.com/respuesta")
respuesta.json()

{'método': 'GET', 'texto': 'Puedes usar esa librería :D'}

In [3]:
respuesta = requests.get("https://profesor-flask.onrender.com/preguntas")
respuesta.json()

{'método': 'GET', 'texto': 'Llevamos: 0 preguntas respondidas'}

In [4]:
respuesta = requests.post("https://profesor-flask.onrender.com/preguntas")
respuesta.json()

{'método': 'POST', 'texto': 'Llevamos: 1 preguntas respondidas'}

# Probemos el codigo!

Finalmente con todo correctamente configurado es hora de probar nuestro codigo, para ello debes ubicarte en la ruta de la carpeta `propuesto`, en esta debes abrir 4 terminales y ejutar los siguientes codigos en el orden establecido:

1. `python3.11 profesor.py`

2. `python3.11 tareo.py`

3. `python3.11 docencio.py`

4. `python3.11 estudiante.py`